在此项目中，我将在PyTorch中构建一个简单的全连接神经网络，并使用MNIST数据集训练它识别手写数字。

1.1 导入python包

In [1]:
from torchvision import transforms
import numpy as np
import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from torch import optim

1.2 加载数据集
    数据位置：torchvision.datasets.MNIST

1.2.1 使用代码加载MNIST数据集

In [2]:
train_batch_size = 64  #设置训练数据集大小为64
test_batch_size = 1000 #设置测试数据集大小为1000
img_size = 28          #设置图像大小为28

def get_dataloader(train=True):

    #准备数据集，其中0.1307，0.3081为MNIST数据的均值和标准差，这样操作能够对其进行标准化
    #因为MNIST只有一个通道（黑白图片）,所以元组中只有一个值
    dataset = torchvision.datasets.MNIST('/data', train=train, download=True,
                                         transform=torchvision.transforms.Compose([
                                         torchvision.transforms.ToTensor(),
                                         torchvision.transforms.Normalize((0.1307,), (0.3081,)),]))
    #准备数据迭代器
    batch_size = train_batch_size if train else test_batch_size
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
    return dataloader

In [3]:
dataset = torchvision.datasets.MNIST('/data', train=True, download=True,
                                         transform=torchvision.transforms.Compose([
                                         torchvision.transforms.ToTensor(),
                                         torchvision.transforms.Normalize((0.1307,), (0.3081,)),]))
# 查看数据构成
dataset.data.size()

torch.Size([60000, 28, 28])

In [6]:
# 查看单个数据，28x28像素构成的图片
dataset.data.size(),dataset.data[1]

(torch.Size([60000, 28, 28]),
 tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,  51, 159, 253, 159,  50,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           48, 238, 252, 252, 252, 237,   0,   0,   0,   0,   0,   0,   0,   0],
    

In [9]:
# 显示图像
from PIL import Image
Image.fromarray(dataset.data[1].numpy())

1.3 构建模型
本项目采用relu作为激活函数

In [10]:
# 测试relu
test_relu_data = torch.tensor([-2, -1,  0,  1,  2])
F.relu(test_relu_data)

tensor([0, 0, 0, 1, 2])

1.3.1 神经网络模型
本项目采用输入层 隐藏层 输出层 三层神经网络结构

In [ ]:
class MnistNet(torch.nn.Module):
    def __init__(self):
        super(MnistNet,self).__init__()
        self.fc1 = torch.nn.Linear(28*28,28)
        self.fc2 = torch.nn.Linear(28,10)

    def forward(self,x):
        # 输入层
        x = x.view(-1,28*28*1)#[batch_size,28*28]
        # 隐藏层 
        x = self.fc1(x) #[batch_size,28]
        # 隐藏层的激活函数
        x = F.relu(x)  #[batch_size,28]
        # 输出层
        x = self.fc2(x) #[batch_size,10]
        # 输出层的激活函数
        out = F.log_softmax(x,dim=-1) #[batch_size,10]
        return out

In [ ]:
my_data = dataset.data[:64]
my_data.size()
x = my_data.view(-1, 28*28*1)
x.size()

1.3 损失函数
本项目损失函数为softmax函数